In [3]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import glob
import pickle
import copy
import random
import time
import traceback
from multiprocessing import Process

# Make file index dict

In [4]:
7323+588

7911

In [2]:
OPT_INDEX = '/Lun0/zhiyong/libri'
# data_dir = '/Lun0/zhiyong/dataset/vox2/dev/aac'
# data_dir_1 = '/Lun0/zhiyong/dataset/vox1/dev/wav'
# data_dir_2test = '/Lun0/zhiyong/dataset/vox2/test/aac'
data_dir_100 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100'
data_dir_360 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360'
data_dir_o500 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-other-500'
new_start_id = 7911 #vox with 0-7322 in with 7323-7910


spk2id = os.path.join(OPT_INDEX, 'spk2id')
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
# spk2utt_val_dict = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
# val_utt_num = 200

In [18]:
spk2utt = {}
# spk2utt_train = {}
# spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

label_dir = glob.glob(data_dir_100+'/*')
label = [i.split('/')[-1] for i in label_dir]

label_dir = glob.glob(data_dir_360+'/*')
label_1 = [i.split('/')[-1] for i in label_dir]

label_dir = glob.glob(data_dir_o500+'/*')
label_2 = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label):
    with open(spk2id, 'a') as f:
        f.write(str(count+new_start_id)+','+i+'\n')
    spk2utt[count+new_start_id] = glob.glob(os.path.join(data_dir_100, i)+'/*/*.flac')

for count_1, i in enumerate(label_1):
    with open(spk2id, 'a') as f:
        f.write(str(new_start_id+count+1+count_1)+','+i+'\n')
    spk2utt[new_start_id+count+1+count_1] = glob.glob(os.path.join(data_dir_360, i)+'/*/*.flac')

for count_2, i in enumerate(label_2):
    with open(spk2id, 'a') as f:
        f.write(str(new_start_id+count+1+count_1+1+count_2)+','+i+'\n')
    spk2utt[new_start_id+count+1+count_1+1+count_2] = glob.glob(os.path.join(data_dir_o500, i)+'/*/*.flac')

In [4]:
# should be class:
251+921+1166

2338

In [7]:
7910 + 2338

10248

In [13]:
len(spk2utt)

2338

In [14]:
spk2utt.keys()

dict_keys([7911, 7912, 7913, 7914, 7915, 7916, 7917, 7918, 7919, 7920, 7921, 7922, 7923, 7924, 7925, 7926, 7927, 7928, 7929, 7930, 7931, 7932, 7933, 7934, 7935, 7936, 7937, 7938, 7939, 7940, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7953, 7954, 7955, 7956, 7957, 7958, 7959, 7960, 7961, 7962, 7963, 7964, 7965, 7966, 7967, 7968, 7969, 7970, 7971, 7972, 7973, 7974, 7975, 7976, 7977, 7978, 7979, 7980, 7981, 7982, 7983, 7984, 7985, 7986, 7987, 7988, 7989, 7990, 7991, 7992, 7993, 7994, 7995, 7996, 7997, 7998, 7999, 8000, 8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008, 8009, 8010, 8011, 8012, 8013, 8014, 8015, 8016, 8017, 8018, 8019, 8020, 8021, 8022, 8023, 8024, 8025, 8026, 8027, 8028, 8029, 8030, 8031, 8032, 8033, 8034, 8035, 8036, 8037, 8038, 8039, 8040, 8041, 8042, 8043, 8044, 8045, 8046, 8047, 8048, 8049, 8050, 8051, 8052, 8053, 8054, 8055, 8056, 8057, 8058, 8059, 8060, 8061, 8062, 8063, 8064, 8065, 8066, 8067, 8068, 8069, 8070, 8071, 8072, 8073, 8074, 8075,

In [19]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

281241


In [22]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)
spk2utt_train = spk2utt

spk2utt_train = copy.deepcopy(spk2utt)

In [23]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

281241


In [24]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [25]:
spk2utt_train_len = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# spk2utt_val_len = os.path.join(OPT_INDEX, 'spk2utt_val_len')

spk2utt_train_len_dict = {}
# spk2utt_val_len_dict = {}

In [26]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

for i in spk2utt_train:
    for utt in spk2utt_train[i]:
        spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

In [49]:
# trial dict
trial_dir = '/Lun0/zhiyong/dataset/vox1/test/wav'
trial_dict_out = os.path.join(OPT_INDEX, 'trial_dict')
trial_dict = {}

tmp_dir_list = glob.glob(trial_dir+'/*/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    trial_dict[count] = this_dir
with open(trial_dict_out, 'wb') as handle:
    pickle.dump(trial_dict, handle)

In [52]:
len(trial_dict)

4874

# make small 100 train

In [4]:
from preprocessing import VoxIterableDataset

In [5]:
OPT_INDEX = '/Lun0/zhiyong/libri_small'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/libri'
data_dir_dict = {}

# train
# data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [6]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [7]:
with open(os.path.join(OPT_INDEX_2, 'spk2utt_train_dict'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [9]:
spk2utt_train_dict.keys()

dict_keys([7911, 7912, 7913, 7914, 7915, 7916, 7917, 7918, 7919, 7920, 7921, 7922, 7923, 7924, 7925, 7926, 7927, 7928, 7929, 7930, 7931, 7932, 7933, 7934, 7935, 7936, 7937, 7938, 7939, 7940, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7953, 7954, 7955, 7956, 7957, 7958, 7959, 7960, 7961, 7962, 7963, 7964, 7965, 7966, 7967, 7968, 7969, 7970, 7971, 7972, 7973, 7974, 7975, 7976, 7977, 7978, 7979, 7980, 7981, 7982, 7983, 7984, 7985, 7986, 7987, 7988, 7989, 7990, 7991, 7992, 7993, 7994, 7995, 7996, 7997, 7998, 7999, 8000, 8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008, 8009, 8010, 8011, 8012, 8013, 8014, 8015, 8016, 8017, 8018, 8019, 8020, 8021, 8022, 8023, 8024, 8025, 8026, 8027, 8028, 8029, 8030, 8031, 8032, 8033, 8034, 8035, 8036, 8037, 8038, 8039, 8040, 8041, 8042, 8043, 8044, 8045, 8046, 8047, 8048, 8049, 8050, 8051, 8052, 8053, 8054, 8055, 8056, 8057, 8058, 8059, 8060, 8061, 8062, 8063, 8064, 8065, 8066, 8067, 8068, 8069, 8070, 8071, 8072, 8073, 8074, 8075,

In [10]:
spk2utt_train_dict_small_100 = {}
spk2utt_val_dict_small_100 = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 100:
        spk2utt_train_dict_small_100[1211+100+count] = spk2utt_train_dict[i]
    else:
        break

In [11]:
spk2utt_train_dict_small_100.keys()

dict_keys([1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410])

In [14]:
spk2utt_train_dict_small_100[1311]

['/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0039.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0029.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0023.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0011.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0020.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0031.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0019.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0008.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0000.flac',
 

In [15]:
for i in spk2utt_train_dict_small_100:
    spk2utt_val_dict_small_100[i] = [spk2utt_train_dict_small_100[i][0]]
    spk2utt_train_dict_small_100[i].pop(0)

In [16]:
spk2utt_train_dict_small_100[1311]

['/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0029.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0023.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0011.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0020.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0031.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0019.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0008.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0000.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0016.flac',
 

In [17]:
spk2utt_val_dict_small_100[1311]

['/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0039.flac']

In [18]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_100'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_100, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_100'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_100, handle)

In [8]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_100')

In [9]:
100 * 200

20000

In [10]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/libri_small/train_data_small_100/'
config['err_log'] = '/Lun0/zhiyong/libri_small/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open(config['err_log'], 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
5336.557748794556


# Make train index

In [11]:
100*200

20000

In [12]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_100')
expected_len = 20000
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_100.csv')

In [13]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [15]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [16]:
count

20000

# Make val data

In [17]:
OPT_INDEX = '/Lun0/zhiyong/libri_small'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/libri'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_100')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [18]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [19]:
config = {}

config['sr'] = 16000
config['repeats'] = 10
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/libri_small/val_data_small_100/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1.9493794441223145
0.1472160816192627
2.1668472290039062
0.09633469581604004
1.0840725898742676
0.20134687423706055
0.06517481803894043
0.37467265129089355
0.618483304977417
0.4271242618560791
0.7301383018493652
1.2740097045898438
0.13498806953430176
1.7492423057556152
0.2702963352203369
0.1591639518737793
0.245255708694458
0.2501842975616455
0.2141554355621338
0.20422816276550293
0.11067056655883789
0.09252500534057617
0.2590444087982178
2.489527463912964
0.01673603057861328
0.3989064693450928
0.13121318817138672
0.08590435981750488
0.11895251274108887
0.3818535804748535
0.7721292972564697
0.4063429832458496
0.3761420249938965
0.17709136009216309
0.3633441925048828
0.23535776138305664
0.4776759147644043
0.42967677116394043
0.3408389091491699
0.4365077018737793
1.0087029933929443
0.46483850479125977
0.2659800052642822
0.6076467037200928
1.015106

0.02679586410522461
0.1194605827331543
0.031792640686035156
0.03131103515625
0.022940874099731445
0.03135490417480469
0.03787040710449219
0.04011845588684082
0.03139781951904297
0.023972511291503906
0.027984142303466797
0.029980897903442383
0.03714919090270996
0.0999755859375
0.04640603065490723
0.05076885223388672
0.014695167541503906
0.04220175743103027
0.014332771301269531
0.07985949516296387
0.05706429481506348
0.03860831260681152
0.03653383255004883
0.03165912628173828
0.02417778968811035
0.038344621658325195
0.07824277877807617
0.04417061805725098
0.0447840690612793
0.024441957473754883
0.014752388000488281
0.03224015235900879
0.02858877182006836
0.04636788368225098
0.07311534881591797
0.038314104080200195
0.030902385711669922
0.016265869140625
0.024845361709594727
0.01503133773803711
0.08002710342407227
0.03148961067199707
0.05672335624694824
0.028914690017700195
0.03384280204772949
0.015342950820922852
0.03253579139709473
0.03007364273071289
0.09215259552001953
0.01866126060485

0.046471595764160156
0.014977693557739258
0.03162980079650879
0.12236452102661133
0.07726788520812988
0.3005228042602539
0.06413578987121582
0.04004669189453125
0.13530993461608887
0.08808064460754395
0.0826730728149414
0.05750584602355957
0.044684648513793945
0.04787445068359375
0.029231786727905273
0.04411888122558594
0.030201196670532227
0.016374588012695312
0.02292943000793457
0.029216289520263672
0.052854299545288086
0.03771829605102539
0.024522066116333008
0.026979923248291016
0.01815009117126465
0.03405261039733887
0.017261505126953125
0.010199308395385742
0.052629709243774414
0.020192861557006836
0.03393387794494629
0.021744728088378906
0.08060860633850098
0.01043701171875
0.0511012077331543
0.08885407447814941
0.010802268981933594
0.030781269073486328
0.033545732498168945
0.07203412055969238
0.03822684288024902
0.040117740631103516
0.0712125301361084
0.046967506408691406
0.0502164363861084
0.05108141899108887
0.07343125343322754
0.01986980438232422
0.02224874496459961
0.019271

# Make validation index (Need only once)

In [20]:
OPT_INDEX = '/Lun0/zhiyong/libri_small'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_small_100')
expected_len = 100 * 10
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_small_100.csv')

In [21]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [22]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [23]:
count

1000

# libri make trial

In [29]:
OPT_INDEX = '/Lun0/zhiyong/libri_small'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/libri'

data_dir_dev_100 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/LibriSpeech/dev-clean'
# data_dir_360 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360'
# data_dir_o500 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-other-500'
new_start_id = 0 #vox with 0-7322 in with 7323-7910


spk2id_dev_dir = os.path.join(OPT_INDEX, 'spk2id')
# spk2utt_trial_dict_small = os.path.join(OPT_INDEX, 'spk2utt_dict')
# spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
# spk2utt_val_dict = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
# val_utt_num = 200

In [ ]:
spk2utt_trial_dict_small

In [30]:
spk2utt_trial_dict_small = {}
# spk2utt_train = {}
# spk2utt_val = {}

with open(spk2id_dev_dir, 'w') as f:
    pass

label_dir = glob.glob(data_dir_dev_100+'/*')
label = [i.split('/')[-1] for i in label_dir]

# label_dir = glob.glob(data_dir_360+'/*')
# label_1 = [i.split('/')[-1] for i in label_dir]

# label_dir = glob.glob(data_dir_o500+'/*')
# label_2 = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label):
    with open(spk2id_dev_dir, 'a') as f:
        f.write(str(count+new_start_id)+','+i+'\n')
    spk2utt_trial_dict_small[count+new_start_id] = glob.glob(os.path.join(data_dir_dev_100, i)+'/*/*.flac')

# for count_1, i in enumerate(label_1):
#     with open(spk2id, 'a') as f:
#         f.write(str(new_start_id+count+1+count_1)+','+i+'\n')
#     spk2utt[new_start_id+count+1+count_1] = glob.glob(os.path.join(data_dir_360, i)+'/*/*.flac')

# for count_2, i in enumerate(label_2):
#     with open(spk2id, 'a') as f:
#         f.write(str(new_start_id+count+1+count_1+1+count_2)+','+i+'\n')
#     spk2utt[new_start_id+count+1+count_1+1+count_2] = glob.glob(os.path.join(data_dir_o500, i)+'/*/*.flac')

In [33]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'wb') as handle:
    pickle.dump(spk2utt_trial_dict_small, handle)

In [35]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'rb') as handle:
    spk2utt_trial_dict_small = pickle.load(handle)

In [36]:
count = 0
for i in spk2utt_trial_dict_small:
    count += len(spk2utt_trial_dict_small[i])
print(count)

2703


In [40]:
OPT_INDEX = '/Lun0/zhiyong/libri_small'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/libri'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [41]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [ ]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None
config['out_dir'] = '/Lun0/zhiyong/libri_small/trial_data_small/'

trial_dict_dir = os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small')

def trial_data_preload(dataset, i, trial_dict_dir):
    with open(trial_dict_dir, 'rb') as handle:
        trial_dict = pickle.load(handle)
        
    start_time = time.time()
    count = 0
    for j in trial_dict:
        for line in trial_dict[j]:
            data = dataset.process_one_utt(line)
            label = line.split('/')[-1][:-4]
            with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), [label]), handle)
            print(time.time()-start_time)
            start_time = time.time()
            count += 1

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, trial_dict_dir)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

43.087857484817505
0.7353620529174805
0.8445770740509033
0.5387043952941895
0.17685174942016602
0.5317568778991699
0.41222357749938965
0.5361981391906738
0.14995908737182617
0.3926863670349121
0.330230712890625
0.056999921798706055
0.26259398460388184
0.15417838096618652
0.38944077491760254
0.2948760986328125
0.28157997131347656
0.20633554458618164
0.22025418281555176
0.13025689125061035
0.32529234886169434
0.5086073875427246
0.2940826416015625
0.20425105094909668
1.470012903213501
0.4965996742248535
0.26575779914855957
0.21291136741638184
0.26731371879577637
0.1757361888885498
0.3464181423187256
0.46500611305236816
0.334531307220459
0.25807976722717285
0.0904076099395752
0.12866544723510742
0.5363068580627441
0.36139488220214844
1.1917800903320312
0.47907137870788574
1.1305527687072754
0.4764258861541748
0.24843549728393555
0.22966432571411133
0.4333512783050537
0.3949723243713379
0.4406557083129883
0.477832555770874
0.19431638717651367
0.3695847988128662
0.6112034320831299
0.36061787

0.33687710762023926
0.42070794105529785
0.3279404640197754
0.3054080009460449
0.15344572067260742
0.17493057250976562
0.39887189865112305
0.3357679843902588
0.418292760848999
0.24967074394226074
0.5587401390075684
0.2340548038482666
0.2993755340576172
0.2967641353607178
0.12192344665527344
0.17511296272277832
0.34916162490844727
0.2487642765045166
0.11678695678710938
0.41438770294189453
0.5060019493103027
0.2537250518798828
0.11202216148376465
0.41463661193847656
0.21437621116638184
0.10650420188903809
0.19287371635437012
0.17525506019592285
0.3875908851623535
0.2648346424102783
0.04616856575012207
0.1635456085205078
0.2534980773925781
0.1599729061126709
0.6298224925994873
0.32042717933654785
0.31952428817749023
1.378605604171753
0.500626802444458
0.11481904983520508
0.3610501289367676
0.43586230278015137
0.12504363059997559
0.7000470161437988
0.8726465702056885
0.28121399879455566
0.2745392322540283
0.1482076644897461
0.19965577125549316
0.26062560081481934
0.13588929176330566
0.44398

# make trial index

In [43]:
train_data_dir = os.path.join(OPT_INDEX, 'trial_data_small')
expected_len = 2703
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'trial_data_small_csv.csv')

In [44]:
len(glob.glob(train_data_dir+'/*'))

14490

In [45]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [47]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [48]:
count

14490

# make trial list

In [59]:
trial_list_out_dir = os.path.join(OPT_INDEX, 'small_tiral_list')
trial_num_per_utt = 10

In [58]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'rb') as handle:
    spk2utt_trial_dict_small = pickle.load(handle)

In [119]:
count = 0
with open(trial_list_out_dir, 'w') as of:
    for i in spk2utt_trial_dict_small:
        for line in spk2utt_trial_dict_small[i]:
            for c in range(trial_num_per_utt):
                if (c % 2) == 0:
                    tmp_list = copy.deepcopy(spk2utt_trial_dict_small[i])
                    tmp_list.remove(line)
                    label_t = random.choice(tmp_list)
                    label_t = label_t.split('/')[-1]
                    label_e = line.split('/')[-1]
                    of.write(label_e+' '+label_t+' '+'target'+'\n')
                    count += 1
                else:
                    tmp_list = list(spk2utt_trial_dict_small.keys())
                    tmp_list.remove(i)
                    spk_t = random.choice(tmp_list)
                    label_t = random.choice(spk2utt_trial_dict_small[spk_t])
                    label_t = label_t.split('/')[-1]
                    label_e = line.split('/')[-1]
                    of.write(label_e+' '+label_t+' '+'nontarget'+'\n')                    
                    count += 1
print(count)

115920
